<a href="https://colab.research.google.com/github/anumsajid13/Fabricae-FYP/blob/main/ImageToImage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clone the required repository and install dependencies
!git clone https://github.com/ostris/ai-toolkit
!pip install diffusers transformers torch psutil
!mkdir -p /content/dataset

# Reasoning:
# - Cloning the repository and installing required libraries (Diffusers, Transformers, Torch, etc.)
# - Creating a directory for the dataset if it doesn't already exist
#   (this will be useful when mounting or downloading data to this environment)

Cloning into 'ai-toolkit'...
remote: Enumerating objects: 3980, done.
remote: Counting objects: 100% (2064/2064), done.
remote: Compressing objects: 100% (245/245), done.
remote: Total 3980 (delta 1947), reused 1839 (delta 1815), pack-reused 1916 (from 1)
Receiving objects: 100% (3980/3980), 29.68 MiB | 15.56 MiB/s, done.
Resolving deltas: 100% (3031/3031), done.


In [ ]:
!cd ai-toolkit && git submodule update --init --recursive && pip install -r requirements.txt

Submodule 'repositories/batch_annotator' (https://github.com/ostris/batch-annotator) registered for path 'repositories/batch_annotator'
Submodule 'repositories/ipadapter' (https://github.com/tencent-ailab/IP-Adapter.git) registered for path 'repositories/ipadapter'
Submodule 'repositories/leco' (https://github.com/p1atdev/LECO) registered for path 'repositories/leco'
Submodule 'repositories/sd-scripts' (https://github.com/kohya-ss/sd-scripts.git) registered for path 'repositories/sd-scripts'
Cloning into '/content/ai-toolkit/repositories/batch_annotator'...
Cloning into '/content/ai-toolkit/repositories/ipadapter'...
Cloning into '/content/ai-toolkit/repositories/leco'...
Cloning into '/content/ai-toolkit/repositories/sd-scripts'...
Submodule path 'repositories/batch_annotator': checked out '420e142f6ad3cc14b3ea0500affc2c6c7e7544bf'
Submodule 'repositories/controlnet' (https://github.com/lllyasviel/ControlNet-v1-1-nightly.git) registered for path 'repositories/batch_annotator/repositor

In [ ]:
import getpass
import os

# Set up Hugging Face Token for accessing pre-trained models
hf_token = getpass.getpass('Enter your Hugging Face access token and press enter: ')
os.environ['HF_TOKEN'] = hf_token
print("HF_TOKEN environment variable has been set.")

# Reasoning:
# - Using Hugging Face's token allows you to load and save models from your Hugging Face account.
# - Setting it as an environment variable enables secure access to protected resources.

Enter your Hugging Face access token and press enter: ··········
HF_TOKEN environment variable has been set.


In [ ]:

# Import torch library
import torch

# Define paths for dataset folders
sketch_folder = '/content/drive/MyDrive/Create_Dataset/Create_Dataset/sketches'
image_folder = '/content/drive/MyDrive/Create_Dataset/Create_Dataset/images'
prompt_folder = '/content/drive/MyDrive/Create_Dataset/Create_Dataset/prompts'

# Define hyperparameters
batch_size = 4
num_epochs = 5
learning_rate = 5e-5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Reasoning:
# - Specifying paths makes the code modular and easy to modify if paths change.
# - Setting hyperparameters (batch size, epochs, learning rate) and selecting the device (GPU or CPU)
#   is essential for training configurations, especially for fine-tuning.



In [ ]:
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

class SketchToImageDataset(Dataset):
    def __init__(self, sketch_folder, image_folder, prompt_folder, transform=None):
        self.sketch_folder = sketch_folder
        self.image_folder = image_folder
        self.prompt_folder = prompt_folder
        self.transform = transform or transforms.ToTensor()

        # Get file lists and base names (i.e., filenames without extensions)
        sketch_files = sorted(os.listdir(sketch_folder))
        image_files = sorted(os.listdir(image_folder))
        prompt_files = sorted(os.listdir(prompt_folder))

        # Get base names (i.e., file names without extensions)
        sketch_base_names = {os.path.splitext(file)[0] for file in sketch_files}
        image_base_names = {os.path.splitext(file)[0] for file in image_files}
        prompt_base_names = {os.path.splitext(file)[0] for file in prompt_files}

        # Find common base names across all three folders
        common_base_names = sketch_base_names.intersection(image_base_names, prompt_base_names)

        # Filter files in each folder that match the common base names
        self.sketch_files = [file for file in sketch_files if os.path.splitext(file)[0] in common_base_names]
        self.image_files = [file for file in image_files if os.path.splitext(file)[0] in common_base_names]
        self.prompt_files = [file for file in prompt_files if os.path.splitext(file)[0] in common_base_names]

    def __len__(self):
        return len(self.sketch_files)

    def __getitem__(self, idx):
      # Get file paths
      sketch_path = os.path.join(self.sketch_folder, self.sketch_files[idx])
      image_path = os.path.join(self.image_folder, self.image_files[idx])
      prompt_path = os.path.join(self.prompt_folder, self.prompt_files[idx])

      # Debugging step to print file paths
      print(f"Loading files: {sketch_path}, {image_path}, {prompt_path}")

      # Ensure the image files exist
      if not os.path.exists(sketch_path):
          raise FileNotFoundError(f"Sketch file not found: {sketch_path}")
      if not os.path.exists(image_path):
          raise FileNotFoundError(f"Image file not found: {image_path}")
      if not os.path.exists(prompt_path):
          raise FileNotFoundError(f"Prompt file not found: {prompt_path}")

      # Load images using PIL
      try:
          sketch_image = Image.open(sketch_path).convert("RGB")
          target_image = Image.open(image_path).convert("RGB")
      except Exception as e:
          print(f"Error loading images: {e}")
          return None  # Handle image loading failure gracefully

      # Debugging step to print image sizes
      print(f"Loaded sketch image: {sketch_image.size}, Loaded target image: {target_image.size}")

      # Load the text prompt
      with open(prompt_path, 'r') as f:
          prompt = f.read().strip()

      # Apply transformation to convert PIL images to tensors
      if self.transform:
          sketch_image = self.transform(sketch_image)
          target_image = self.transform(target_image)

      return {'sketch': sketch_image, 'image': target_image, 'prompt': prompt}



# Define the transformations (convert images to tensors)
transform = transforms.Compose([
    transforms.Resize((512, 512)),  # Resize to a consistent size
    transforms.ToTensor(),  # Convert images to tensors
])

# Initialize the dataset with the transformations
dataset = SketchToImageDataset(sketch_folder, image_folder, prompt_folder, transform=transform)


Mounted at /content/drive


In [ ]:
from torch.utils.data import DataLoader
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

import os

# Function to get base names (without file extensions)
def get_base_names(file_list):
    return {os.path.splitext(file)[0] for file in file_list}

# List files in each folder
sketch_files = os.listdir(sketch_folder)
image_files = os.listdir(image_folder)
prompt_files = os.listdir(prompt_folder)

# Get base names (i.e., file names without extensions)
sketch_base_names = get_base_names(sketch_files)
image_base_names = get_base_names(image_files)
prompt_base_names = get_base_names(prompt_files)

# Find common base names across all three folders
common_base_names = sketch_base_names.intersection(image_base_names, prompt_base_names)

# Filter files in each folder that match the common base names
sketch_files_filtered = [file for file in sketch_files if os.path.splitext(file)[0] in common_base_names]
image_files_filtered = [file for file in image_files if os.path.splitext(file)[0] in common_base_names]
prompt_files_filtered = [file for file in prompt_files if os.path.splitext(file)[0] in common_base_names]

# Display the filtered file names
print("Filtered Sketch Files:", sketch_files_filtered)
print("Filtered Image Files:", image_files_filtered)
print("Filtered Prompt Files:", prompt_files_filtered)

# Store the filtered files in respective variables
sketch_files = sketch_files_filtered
image_files = image_files_filtered
prompt_files = prompt_files_filtered

print("Number of Sketch Files:", len(sketch_files))
print("Number of Image Files:", len(image_files))
print("Number of Prompt Files:", len(prompt_files))

# Initialize dataset and dataloader with transformations
dataset = SketchToImageDataset(sketch_folder, image_folder, prompt_folder, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Reasoning:
# - Initializing the dataset instance and using DataLoader to batch data and shuffle it.
# - Batching is necessary for efficient GPU processing, and shuffling adds randomness for better training.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Filtered Sketch Files: ['image1.png', 'image100.png', 'image103.png', 'image105.png', 'image107.png', 'image108.png', 'image114.png', 'image12.png', 'image123.png', 'image125.png', 'image13.png', 'image130.png', 'image131.png', 'image134.png', 'image139.png', 'image14.png', 'image140.png', 'image141.png', 'image146.png', 'image147.png', 'image148.png', 'image152.png', 'image158.png', 'image159.png', 'image160.png', 'image163.png', 'image167.png', 'image176.png', 'image18.png', 'image183.png', 'image198.png', 'image2.png', 'image21.png', 'image212.png', 'image213.png', 'image219.png', 'image220.png', 'image226.png', 'image229.png', 'image23.png', 'image231.png', 'image233.png', 'image234.png', 'image235.png', 'image236.png', 'image24.png', 'image240.png', 'image242.png', 'image246.png', 'image25.png', 'image254.png', 'image258.png', 'image259.png', 'image262.p

In [ ]:
from diffusers import StableDiffusionXLControlNetPipeline, ControlNetModel,AutoencoderKL
import torch

# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the base model (Stable Diffusion) and the ControlNet model
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
controlnet = ControlNetModel.from_pretrained("xinsir/controlnet-scribble-sdxl-1.0", torch_dtype=torch.float16)
pipeline = StableDiffusionXLControlNetPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    controlnet=controlnet,
    vae=vae,
    torch_dtype=torch.float16
)

# Move pipeline to device
pipeline = pipeline.to(device)

# Enable slicing to save memory
pipeline.vae.enable_slicing()

# Reasoning:
# - By specifying `controlnet=controlnet`, we ensure that the pipeline receives all expected components.
# - Moving to `float16` and enabling slicing helps reduce memory usage, making the pipeline more memory-efficient.



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/10.3G [00:00<?, ?B/s]

tokenizer_2/tokenizer_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

unet/config.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
from transformers import CLIPTokenizer

# Load tokenizer for text prompts
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")

# Reasoning:
# - Tokenizer processes text prompts, converting them into a form that the model can interpret.
# - We use the CLIP tokenizer as it matches the text encoder in the model.

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

In [ ]:
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [ ]:
from torch.optim import AdamW

# Define the components to fine-tune
params_to_optimize = list(pipeline.unet.parameters()) + list(pipeline.text_encoder.parameters()) + list(pipeline.controlnet.parameters())

# Set up optimizer with the selected parameters
optimizer = AdamW(params_to_optimize, lr=learning_rate)

# Reasoning:
# - Only the necessary components (UNet, Text Encoder, and ControlNet) are included in fine-tuning, optimizing memory usage and focus.
# - AdamW is chosen due to its effectiveness in fine-tuning transformer-based architectures, like those in the pipeline.


In [ ]:
from torchvision import transforms
from tqdm import tqdm
# Define a transform to convert tensors to PIL images if needed (but not used here)
# to_pil = transforms.ToPILImage()

pipeline.to(device)

# Training loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    # Set the models to train mode
    pipeline.unet.train()
    pipeline.text_encoder.train()
    pipeline.controlnet.train()

    for batch in tqdm(dataloader, desc="Training"):
        # Move data to device
        sketch_images = batch['sketch'].to(device)
        target_images = batch['image'].to(device)
        prompts = batch['prompt']

        # Tokenize text prompts
        text_inputs = tokenizer(prompts, padding="max_length", return_tensors="pt").to(device)

        # Check if any data is missing
        if sketch_images is None or target_images is None or text_inputs is None:
            print(f"Skipping batch {idx} due to missing data.")
            continue

        # Forward pass
        try:
            outputs = pipeline(
                images=target_images,
                controlnet_cond=sketch_images,
                prompt=prompts
            )
        except TypeError as e:
            print(f"Error during pipeline forward pass: {e}")
            continue

        # Convert the output images to tensors if necessary for loss calculation
        output_images_tensor = outputs.images.to(device)

        # Calculate loss
        loss = torch.nn.functional.mse_loss(output_images_tensor, target_images)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss for each batch
        print(f"Loss: {loss.item()}")

    # Save checkpoint
    pipeline.save_pretrained(f"./model_checkpoint_epoch_{epoch+1}")


Epoch 1/5


Training:   0%|          | 0/158 [00:00<?, ?it/s]

Loading files: /content/drive/MyDrive/Create_Dataset/Create_Dataset/sketches/image289.png, /content/drive/MyDrive/Create_Dataset/Create_Dataset/images/image289.png, /content/drive/MyDrive/Create_Dataset/Create_Dataset/prompts/image289.txt
Loaded sketch image: (512, 512), Loaded target image: (512, 512)
Loading files: /content/drive/MyDrive/Create_Dataset/Create_Dataset/sketches/image514.png, /content/drive/MyDrive/Create_Dataset/Create_Dataset/images/image514.png, /content/drive/MyDrive/Create_Dataset/Create_Dataset/prompts/image514.txt
Loaded sketch image: (512, 512), Loaded target image: (512, 512)
Loading files: /content/drive/MyDrive/Create_Dataset/Create_Dataset/sketches/image256.png, /content/drive/MyDrive/Create_Dataset/Create_Dataset/images/image256.png, /content/drive/MyDrive/Create_Dataset/Create_Dataset/prompts/image256.txt
Loaded sketch image: (512, 512), Loaded target image: (512, 512)
Loading files: /content/drive/MyDrive/Create_Dataset/Create_Dataset/sketches/image167.pn

Training:   1%|          | 1/158 [00:10<27:08, 10.37s/it]

Error during pipeline forward pass: image must be passed and be one of PIL image, numpy array, torch tensor, list of PIL images, list of numpy arrays or list of torch tensors, but is <class 'NoneType'>
Loading files: /content/drive/MyDrive/Create_Dataset/Create_Dataset/sketches/image652.png, /content/drive/MyDrive/Create_Dataset/Create_Dataset/images/image652.png, /content/drive/MyDrive/Create_Dataset/Create_Dataset/prompts/image652.txt
Loaded sketch image: (512, 512), Loaded target image: (512, 512)
Loading files: /content/drive/MyDrive/Create_Dataset/Create_Dataset/sketches/image656.png, /content/drive/MyDrive/Create_Dataset/Create_Dataset/images/image656.png, /content/drive/MyDrive/Create_Dataset/Create_Dataset/prompts/image656.txt
Loaded sketch image: (512, 512), Loaded target image: (512, 512)
Loading files: /content/drive/MyDrive/Create_Dataset/Create_Dataset/sketches/image314.png, /content/drive/MyDrive/Create_Dataset/Create_Dataset/images/image314.png, /content/drive/MyDrive/Cr

...

Loaded sketch image: (512, 512), Loaded target image: (512, 512)
Loading files: /content/drive/MyDrive/Create_Dataset/Create_Dataset/sketches/image548.png, /content/drive/MyDrive/Create_Dataset/Create_Dataset/images/image548.png, /content/drive/MyDrive/Create_Dataset/Create_Dataset/prompts/image548.txt
Loaded sketch image: (512, 512), Loaded target image: (512, 512)
Loading files: /content/drive/MyDrive/Create_Dataset/Create_Dataset/sketches/image42.png, /content/drive/MyDrive/Create_Dataset/Create_Dataset/images/image42.png, /content/drive/MyDrive/Create_Dataset/Create_Dataset/prompts/image42.txt
Loaded sketch image: (512, 512), Loaded target image: (512, 512)
Error during pipeline forward pass: image must be passed and be one of PIL image, numpy array, torch tensor, list of PIL images, list of numpy arrays or list of torch tensors, but is <class 'NoneType'>
Loading files: /content/drive/MyDrive/Create_Dataset/Create_Dataset/sketches/image284.png, /content/drive/MyDrive/Create_Dataset

Training:  99%|█████████▉| 157/158 [00:21<00:00,  9.04it/s]

Loaded sketch image: (512, 512), Loaded target image: (512, 512)
Loading files: /content/drive/MyDrive/Create_Dataset/Create_Dataset/sketches/image100.png, /content/drive/MyDrive/Create_Dataset/Create_Dataset/images/image100.png, /content/drive/MyDrive/Create_Dataset/Create_Dataset/prompts/image100.txt
Loaded sketch image: (512, 512), Loaded target image: (512, 512)
Loading files: /content/drive/MyDrive/Create_Dataset/Create_Dataset/sketches/image314.png, /content/drive/MyDrive/Create_Dataset/Create_Dataset/images/image314.png, /content/drive/MyDrive/Create_Dataset/Create_Dataset/prompts/image314.txt
Loaded sketch image: (512, 512), Loaded target image: (512, 512)
Error during pipeline forward pass: image must be passed and be one of PIL image, numpy array, torch tensor, list of PIL images, list of numpy arrays or list of torch tensors, but is <class 'NoneType'>
Loading files: /content/drive/MyDrive/Create_Dataset/Create_Dataset/sketches/image1.png, /content/drive/MyDrive/Create_Datase

Training: 100%|██████████| 158/158 [00:22<00:00,  7.16it/s]


Loaded sketch image: (512, 512), Loaded target image: (512, 512)
Loading files: /content/drive/MyDrive/Create_Dataset/Create_Dataset/sketches/image331.png, /content/drive/MyDrive/Create_Dataset/Create_Dataset/images/image331.png, /content/drive/MyDrive/Create_Dataset/Create_Dataset/prompts/image331.txt
Loaded sketch image: (512, 512), Loaded target image: (512, 512)
Loading files: /content/drive/MyDrive/Create_Dataset/Create_Dataset/sketches/image118.png, /content/drive/MyDrive/Create_Dataset/Create_Dataset/images/image118.png, /content/drive/MyDrive/Create_Dataset/Create_Dataset/prompts/image118.txt
Loaded sketch image: (512, 512), Loaded target image: (512, 512)
Error during pipeline forward pass: image must be passed and be one of PIL image, numpy array, torch tensor, list of PIL images, list of numpy arrays or list of torch tensors, but is <class 'NoneType'>


In [ ]:
print("Fine-tuning complete.")
pipeline.save_pretrained("/content/final_model")
print("Model saved.")

Fine-tuning complete.
Model saved.


In [ ]:
from diffusers import StableDiffusionXLControlNetPipeline, ControlNetModel
from transformers import CLIPTokenizer
import torch
import numpy as np
from PIL import Image

# Load model and tokenizers
model_path = "/content/final_model"
controlnet_path = "xinsir/controlnet-scribble-sdxl-1.0"
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")

# Load ControlNet
controlnet = ControlNetModel.from_pretrained(controlnet_path, torch_dtype=torch.float16)

# Initialize the pipeline with ControlNet
pipeline = StableDiffusionXLControlNetPipeline.from_pretrained(
    model_path,
    tokenizer=tokenizer,
    tokenizer_2=tokenizer,
    controlnet=controlnet,
    torch_dtype=torch.float16  # Use float16 precision
)

# Apply memory optimizations
pipeline.enable_attention_slicing("auto")  # Enable memory efficient attention
pipeline.enable_model_cpu_offload()  # Offload layers to CPU when not in use

# Move pipeline to GPU
pipeline.to("cpu")

# Load and preprocess the sketch image
sketch_image_path = "/content/drive/MyDrive/check1.png"
try:
    sketch_image = Image.open(sketch_image_path).convert("RGB")
    sketch_image = sketch_image.resize((512, 512))
    sketch_image = torch.from_numpy(np.array(sketch_image)).permute(2, 0, 1).unsqueeze(0).to("cuda").half()
except Exception as e:
    raise ValueError(f"Error loading the sketch image: {e}")

# Define the prompt
prompt = "A colored textile pattern that looks similar to the sketch pattern I gave, a pleasing textile pattern."

# Generate the output
with torch.no_grad():
    output = pipeline(
        prompt=prompt,
        image=sketch_image,
        controlnet_conditioning_scale=1.0,
        guidance_scale=7.5,
    )

# Display and save the output image
output_image = output.images[0]
output_image.show()
output_image.save("/content/drive/MyDrive/generated_image.png")
print("Image saved successfully!")


Loading pipeline components...:   0%|          | 0/8 [00:00<?, ?it/s]

It seems like you have activated model offloading by calling `enable_model_cpu_offload`, but are now manually moving the pipeline to GPU. It is strongly recommended against doing so as memory gains from offloading are likely to be lost. Offloading automatically takes care of moving the individual components vae, text_encoder, text_encoder_2, tokenizer, tokenizer_2, unet, controlnet, scheduler, feature_extractor, image_encoder to GPU when needed. To make sure offloading works as expected, you should consider moving the pipeline back to CPU: `pipeline.to('cpu')` or removing the move altogether if you use offloading.


  0%|          | 0/50 [00:00<?, ?it/s]

Image saved successfully!


In [ ]:
# Install necessary packages
!pip install diffusers transformers controlnet-aux --quiet
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --quiet
!pip install opencv-python-headless pillow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.4/282.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 26.7 MB/s eta 0:00:00


In [ ]:
# Import libraries
from diffusers import ControlNetModel, StableDiffusionXLControlNetPipeline, AutoencoderKL, EulerAncestralDiscreteScheduler
from controlnet_aux import HEDdetector
from PIL import Image
import torch
import numpy as np
import cv2
import random
import os
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')


# Function to apply Non-Maximum Suppression (NMS)
def nms(x, t, s):
    x = cv2.GaussianBlur(x.astype(np.float32), (0, 0), s)
    f1 = np.array([[0, 0, 0], [1, 1, 1], [0, 0, 0]], dtype=np.uint8)
    f2 = np.array([[0, 1, 0], [0, 1, 0], [0, 1, 0]], dtype=np.uint8)
    f3 = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]], dtype=np.uint8)
    f4 = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]], dtype=np.uint8)

    y = np.zeros_like(x)
    for f in [f1, f2, f3, f4]:
        np.putmask(y, cv2.dilate(x, kernel=f) == x, x)

    z = np.zeros_like(y, dtype=np.uint8)
    z[y > t] = 255
    return z

# Set up model parameters
controlnet_conditioning_scale = 1.0
prompt = "A high-quality rightly colored detailed version of the sketch, realistic and vibrant like a textle pattern."
negative_prompt = 'lowres, bad anatomy, bad hands, missing fingers, extra digit, low quality'

# Load the scheduler and model
#eulera_scheduler = EulerAncestralDiscreteScheduler.from_pretrained("/content/final_model", subfolder="scheduler")
eulera_scheduler = EulerAncestralDiscreteScheduler.from_pretrained("fyp1/sketchToImage", subfolder="scheduler")

#controlnet = ControlNetModel.from_pretrained("/content/final_model/controlnet", torch_dtype=torch.float16)
controlnet = ControlNetModel.from_pretrained("fyp1/sketchToImage", subfolder="controlnet", torch_dtype=torch.float16)

vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)

pipe = StableDiffusionXLControlNetPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    controlnet=controlnet,
    vae=vae,
    safety_checker=None,
    torch_dtype=torch.float16,
    scheduler=eulera_scheduler,
).to("cpu")

# Load your pencil sketch
uploaded_sketch_path = "/content/drive/MyDrive/image19.png"  # Update with your sketch path
controlnet_img = Image.open(uploaded_sketch_path).convert("L")  # Ensure it's grayscale

# Pre-process sketch
controlnet_img = controlnet_img.resize((1024, 1024))

# Run the pipeline to generate the image from your sketch
images = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    image=controlnet_img,
    controlnet_conditioning_scale=controlnet_conditioning_scale,
    width=1024,
    height=1024,
    num_inference_steps=30,
).images

# Save and display the generated image
output_path = "/content/drive/MyDrive/generated_image.png"
images[0].save(output_path)
images[0].show()

/usr/local/lib/python3.10/dist-packages/controlnet_aux/mediapipe_face/mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


scheduler/scheduler_config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

controlnet/config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/2.50G [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/10.3G [00:00<?, ?B/s]

Keyword arguments {'safety_checker': None} are not expected by StableDiffusionXLControlNetPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argument, or use another device for inference.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argument, or use another device for inference.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, du

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

pipeline.save_pretrained('/content/drive/MyDrive')
print(f"Fine-tuning complete. Model saved to {'/content/drive/MyDrive'}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Fine-tuning complete. Model saved to /content/drive/MyDrive
